In [7]:
import tensorflow as tf
from tensorflow import keras
from skimage.io import imread, imshow
from skimage.transform import resize
from tqdm import tqdm 
#import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

In [8]:
#from crfrnn_layer import CrfRnnLayer
from keras.models import Model
from keras_segmentation.models.model_utils import get_segmentation_model
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, \
    Dropout, Conv2DTranspose, Cropping2D, Add, UpSampling2D
from keras.layers.merge import concatenate

In [9]:
from keras.preprocessing.image import ImageDataGenerator
import glob
from keras.models import *
from keras.layers import *
from keras import backend as keras
from keras import Model

In [10]:
import sys
sys.path.insert(1, './src')
from crfrnn_model import get_crfrnn_model_def
from crfrnn_layer import CrfRnnLayer
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
channels, height, width = 1, 512, 512

n_classes = 2 #21

# Input
input_shape = (height, width, 1)
img_input = Input(shape=input_shape)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(img_input)
conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)

up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
merge6 = concatenate([conv4,up6], axis = 3)
conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)


up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)

up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)

up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

out = Conv2D(2, 1, activation = 'sigmoid')(conv9)

crf_output = CrfRnnLayer(image_dims=(height, width),
                     num_classes=n_classes,
                     theta_alpha=160.,
                     theta_beta=3.,
                     theta_gamma=3.,
                     num_iterations=10,
                     name='crfrnn')([out, img_input])


model = Model(inputs = img_input, outputs = crf_output)

model.compile(optimizer = keras.optimizers.Adam(lr = 1e-4),loss = 'binary_crossentropy', metrics = ['accuracy'])
    
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [12]:
def trainGenerator(batch_size,train_path,image_folder,mask_folder,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (512,512),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(rescale=1.0/255.0)
    mask_datagen = ImageDataGenerator(rescale=1.0/255.0)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        #img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)

In [13]:
data = trainGenerator(1,'/Users/mavaylon/Research/Research_Gambier/Data','gamb_orig','Correct_Labels',save_to_dir = None)

In [14]:
data2 = trainGenerator(1,'/Users/mavaylon/Research/Pytorch_UNet/500x500','orig','gt',target_size = (500,500),save_to_dir = None)

In [ ]:
model.fit(data,steps_per_epoch=256,epochs=2) #segmentation fault

Found 512 images belonging to 1 classes.
Found 512 images belonging to 1 classes.
Epoch 1/2


In [15]:
channels, height, width = 1, 500, 500

n_classes = 2 #21

# Input
input_shape = (height, width, 1)
img_input = Input(shape=input_shape)

# Add plenty of zero padding
x = ZeroPadding2D(padding=(100, 100))(img_input)

# VGG-16 convolution block 1
x = Conv2D(64, (3, 3), activation='relu', padding='valid', name='conv1_1')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)

# VGG-16 convolution block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2', padding='same')(x)

# VGG-16 convolution block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool3', padding='same')(x)
pool3 = x

# VGG-16 convolution block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool4', padding='same')(x)
pool4 = x

# VGG-16 convolution block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool5', padding='same')(x)

# Fully-connected layers converted to convolution layers
x = Conv2D(4096, (7, 7), activation='relu', padding='valid', name='fc6')(x)
x = Dropout(0.5)(x)
x = Conv2D(4096, (1, 1), activation='relu', padding='valid', name='fc7')(x)
x = Dropout(0.5)(x)
x = Conv2D(n_classes, (1, 1), padding='valid', name='score-fr')(x)

# Deconvolution
score2 = Conv2DTranspose(n_classes, (4, 4), strides=2, name='score2')(x)

# Skip connections from pool4
score_pool4 = Conv2D(n_classes, (1, 1), name='score-pool4')(pool4)
score_pool4c = Cropping2D((5, 5))(score_pool4)
score_fused = Add()([score2, score_pool4c])
score4 = Conv2DTranspose(n_classes, (4, 4), strides=2, name='score4', use_bias=False)(score_fused)

# Skip connections from pool3
score_pool3 = Conv2D(n_classes, (1, 1), name='score-pool3')(pool3)
score_pool3c = Cropping2D((9, 9))(score_pool3)

# Fuse things together
score_final = Add()([score4, score_pool3c])

# Final up-sampling and cropping
upsample = Conv2DTranspose(n_classes, (16, 16), strides=8, name='upsample', use_bias=False)(score_final)
upscore = Cropping2D(((31, 37), (31, 37)))(upsample)

output = CrfRnnLayer(image_dims=(height, width),
                     num_classes=n_classes,
                     theta_alpha=160.,
                     theta_beta=3.,
                     theta_gamma=3.,
                     num_iterations=10,
                     name='crfrnn')([upscore, img_input])
model = Model(inputs = img_input, outputs = output)

model.compile(optimizer = keras.optimizers.Adam(lr = 1e-4),loss = 'binary_crossentropy', metrics = ['accuracy'])
    
model.summary()
# Build the model
#model_crfrnn = Model(img_input, output)
#model_crfrnn = get_segmentation_model(img_input, output)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 500, 500, 1) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 700, 700, 1)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 698, 698, 64) 640         zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, 698, 698, 64) 36928       conv1_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.fit(data2,steps_per_epoch=256,epochs=2) #segmentation fault

Found 512 images belonging to 1 classes.
Found 512 images belonging to 1 classes.
Epoch 1/2
